In [1]:
#### DO NOT RUN THIS CODE IN THE NOTEBOOK - copy paste into terminal in the HOME directory!

# git clone https://github.com/tensorflow/models.git

In [2]:
 # !pip install s3fs -q
 # !pip install tensorflow_hub -q
 # !pip install tensorflow_datasets -q
 # !pip install pycocotools -q
 # !pip install gin-config -q
 # !pip install immutabledict -q
 # !pip install sentencepiece -q
 # !pip install transformers -q
 # !pip install -r official/projects/movinet/requirements.txt -q

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import configparser

config = configparser.ConfigParser()
config.read('/home/ec2-user/.aws/credentials')

['/home/ec2-user/.aws/credentials']

In [5]:
# AWS credentials and S3 settings
aws_access_key_id = config["default"]['aws_access_key_id']
aws_secret_access_key = config["default"]['aws_secret_access_key']
bucket_name = 'asl-capstone'
s3_URI = 's3://asl-capstone/'

In [6]:
import s3fs
fs = s3fs.S3FileSystem(key=aws_access_key_id, secret=aws_secret_access_key)

import boto3
s3 = boto3.client('s3',aws_access_key_id = aws_access_key_id, aws_secret_access_key = aws_secret_access_key,
                  region_name = 'us-west-2')


In [7]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import os

from sklearn.model_selection import train_test_split

tf.keras.backend.clear_session()

2023-11-16 07:02:40.155839: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-16 07:02:40.858655: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [8]:
%cd /home/ec2-user/models

/home/ec2-user/models


In [9]:
pwd

'/home/ec2-user/models'

In [10]:
# # !wget https://storage.googleapis.com/tf_model_garden/vision/movinet/movinet_a0_base.tar.gz
# !wget https://storage.googleapis.com/tf_model_garden/vision/movinet/movinet_a3_base.tar.gz
# # !wget https://storage.googleapis.com/tf_model_garden/vision/movinet/movinet_a5_base.tar.gz
# # !wget https://storage.googleapis.com/tf_model_garden/vision/movinet/movinet_a5_base_k400.tar.gz

# # !tar -xf movinet_a5_base_k400.tar.gz
# # !mv movinet_a5_base movinet_a5_base_k400
# # !tar -xf movinet_a0_base.tar.gz
# !tar -xf movinet_a3_base.tar.gz
# # !tar -xf movinet_a5_base.tar.gz

In [11]:
model_version = 'a3'

if model_version=='a3':
  max_frames = 120
  image_dims = (256,256)
elif model_version=='a0':
  max_frames = 50
  image_dims = (172,172)
elif model_version=='a5':
  max_frames = 120
  image_dims = (320,302)

In [12]:
# Get the kinetics-600 action labels
KINETICS_URL = "official/projects/movinet/files/kinetics_600_labels.txt"
with open(KINETICS_URL) as obj:
  labels_600 = [line for line in obj.readlines()]
print("Found %d labels." % len(labels_600))

Found 600 labels.


In [13]:
# # Get the kinetics-400 action labels from the GitHub repository.
# from urllib import request  # requires python3
# KINETICS_URL = "https://raw.githubusercontent.com/deepmind/kinetics-i3d/master/data/label_map.txt"
# with request.urlopen(KINETICS_URL) as obj:
#   labels_400 = [line.decode("utf-8").strip() for line in obj.readlines()]
# print("Found %d labels." % len(labels_400))

In [14]:
# !export XLA_PYTHON_CLIENT_PREALLOCATE=false
# !export XLA_FLAGS="--xla_gpu_strict_conv_algorithm_picker=false --xla_gpu_force_compilation_parallelism=1"

In [16]:
video_files = pd.read_parquet('s3://asl-capstone/wlasl/video_mappings_masterfile.parquet')

In [17]:
display(video_files.head())
print(video_files.shape)
print(video_files.word.nunique())

,rgb_path,padded_path,word,frame_count,label_num
0,asl-capstone/wlasl/RGB/viewpoint_0.npy,asl-capstone/wlasl/padded/viewpoint_0.npy,viewpoint,93,1892
1,asl-capstone/wlasl/RGB/viewpoint_1.npy,asl-capstone/wlasl/padded/viewpoint_1.npy,viewpoint,82,1892
2,asl-capstone/wlasl/RGB/viewpoint_2.npy,asl-capstone/wlasl/padded/viewpoint_2.npy,viewpoint,40,1892
3,asl-capstone/wlasl/RGB/viewpoint_3.npy,asl-capstone/wlasl/padded/viewpoint_3.npy,viewpoint,81,1892
4,asl-capstone/wlasl/RGB/viewpoint_4.npy,asl-capstone/wlasl/padded/viewpoint_4.npy,viewpoint,111,1892


(11938, 5)
1995


In [15]:
NUM_CLASSES = 1995

In [18]:
print(video_files['frame_count'].mean())
print(video_files['frame_count'].median())

68.84184955603953
70.0


In [22]:
train_files, test_files = train_test_split(video_files, train_size=0.8)


In [23]:
def data_generator(file_paths_df):
    for _, row in file_paths_df.iterrows():
        with fs.open(row['padded_path'], 'rb') as f:
            video = np.load(f)

        #video = pad_video_frames(video, n_frames=n_frames)
        label = tf.one_hot(row['label_num'], NUM_CLASSES)
        yield video, label


# Create TensorFlow Dataset for model input. 
train_dataset = tf.data.Dataset.from_generator(
    generator=lambda: data_generator(train_files),
    output_signature=(
        tf.TensorSpec(shape=(120, 256, 256, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(NUM_CLASSES), dtype=tf.int32)
    )
)

# Create TensorFlow Dataset for model input. 
test_dataset = tf.data.Dataset.from_generator(
    generator=lambda: data_generator(test_files),
    output_signature=(
        tf.TensorSpec(shape=(120, 256, 256, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(NUM_CLASSES), dtype=tf.int32)
    )
)

2023-11-16 07:02:42.905739: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-16 07:02:42.925542: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-16 07:02:42.926352: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [24]:
#next(iter(train_dataset.batch(1))) #Does a sanity check to confirm that the dataset is processed in batches

In [25]:
#from official.projects.movinet.modeling import movinet_model

from official.projects.movinet.modeling import movinet

# Create backbone and model.
backbone = movinet.Movinet(
    model_id=model_version, #change to correspond to model
    causal=False,
    use_external_states=False,
)

In [26]:
from official.projects.movinet.modeling import movinet_model_mod

# model_id = 'a3'

# # #tf.keras.backend.clear_session()

# backbone = movinet.Movinet(model_id=model_id)
# model = movinet_model_mod.MovinetClassifier(backbone=backbone, num_classes=600)
# model.build([1, 1, 1, 1, 3])

# # Load pretrained weights
# #!wget https://storage.googleapis.com/tf_model_garden/vision/movinet/movinet_a0_base.tar.gz -O movinet_a0_base.tar.gz -q
# #!tar -xvf movinet_a0_base.tar.gz

# checkpoint_dir = 'movinet_a3_base'
# checkpoint_path = tf.train.latest_checkpoint(checkpoint_dir)
# checkpoint = tf.train.Checkpoint(model=model)
# status = checkpoint.restore(checkpoint_path)
# #status.assert_existing_objects_matched()

def build_classifier(backbone, num_classes, freeze_backbone=False):
  """Builds a classifier on top of a backbone model."""
  model = movinet_model_mod.MovinetClassifier(
      backbone=backbone,
      num_classes=num_classes)
  model.build([2, 120, 256, 256, 3])

  if freeze_backbone:
    for layer in model.layers[:-1]:
      layer.trainable = False
    model.layers[-1].trainable = True

  return model

# Wrap the backbone with a new classifier to create a new classifier head
# with num_classes outputs (101 classes for UCF101).
# Freeze all layers except for the final classifier head.
model = build_classifier(backbone, NUM_CLASSES, freeze_backbone=True)

In [27]:
NUM_EPOCHS = 5
BATCH_SIZE=8

train_steps = len(train_files) // BATCH_SIZE
total_train_steps = train_steps * NUM_EPOCHS

test_steps = len(test_files) // BATCH_SIZE

loss_obj = tf.keras.losses.CategoricalCrossentropy(
    from_logits=True,
    label_smoothing=0.1)

metrics = [
    tf.keras.metrics.TopKCategoricalAccuracy(
        k=1, name='top_1', dtype=tf.float32),
    tf.keras.metrics.TopKCategoricalAccuracy(
        k=5, name='top_5', dtype=tf.float32),
]

metric = tf.keras.metrics.TopKCategoricalAccuracy(k=1, name='top_1', dtype=tf.float32)

initial_learning_rate = 0.01
learning_rate = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate, decay_steps=total_train_steps,
)

model.compile(loss={'classifier_head':loss_obj, 'classifier_head_1':None}, 
              optimizer='RMSprop', 
              metrics={'classifier_head':metrics, 'classifier_head_1':None})

In [ ]:
model.summary()

In [ ]:
with tf.device("/GPU:0"):
    results = model.fit(
                    train_dataset.batch(BATCH_SIZE),
                    validation_data=test_dataset.batch(BATCH_SIZE),
                    epochs=NUM_EPOCHS,
                    steps_per_epoch=train_steps,
                    validation_steps=test_steps,
                    # validation_freq=1,
                    verbose=1)

    # model.fit(train_dataset.batch(8), 
    #           validation_data=test_dataset.batch(8), 
    #           epochs=5)

Epoch 1/5


2023-11-16 07:05:37.342626: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x2025f2f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-16 07:05:37.342680: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A10G, Compute Capability 8.6
2023-11-16 07:05:38.436083: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8903
2023-11-16 07:05:42.146784: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2023-11-16 07:06:21.476887: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-11-16 07:06:21.482751: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


  34/1193 [..............................] - ETA: 22:32:23 - loss: 19.3139 - classifier_head_loss: 19.3139 - classifier_head_top_1: 0.0000e+00 - classifier_head_top_5: 9.6982e-04